### DeepFM :https://github.com/shenweichen/DeepCTR-Torch
### Paper : https://www.ijcai.org/proceedings/2017/0239.pdf
 * DeepFM do not need domain knowledge because this model Don't use feature engineering
 * So We run it!

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import gc

plt.style.use('ggplot')

import warnings as w
w.filterwarnings(action='ignore')
pd.set_option('display.max_columns',None)

In [2]:
dtype={'id': np.dtype(int),
    'click': np.dtype(int),
    'hour': np.str,
    'C1': np.dtype(str),
    'banner_pos': np.dtype(str),
    'site_id': np.dtype(str),
    'site_domain': np.dtype(str), 
    'site_category': np.dtype(str),
    'app_id': np.dtype(str),
    'app_domain': np.dtype(str),
    'app_category': np.dtype(str),
    'device_id': np.dtype(str),
    'device_ip': np.dtype(str),
    'device_model': np.dtype(str),
    'device_type': np.dtype(str),
    'device_conn_type': np.dtype(str),
    'C14': np.dtype(str),
    'C15': np.dtype(str),
    'C16': np.dtype(str),
    'C17': np.dtype(str),
    'C18': np.dtype(str),
    'C19': np.dtype(str),
    'C20': np.dtype(str),
    'C21':np.dtype(str)
      }
num_records = 40428967
sample_size = 500000
skip_values = sorted(random.sample(range(1,num_records), num_records - sample_size))
parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')

In [3]:
train = pd.read_csv("avazu-ctr-prediction/train.gz", parse_dates=['hour'], date_parser=parse_date, dtype=dtype, skiprows=skip_values)
test = pd.read_csv("avazu-ctr-prediction/test.gz", parse_dates=['hour'], date_parser=parse_date, dtype=dtype)
submission = pd.read_csv('avazu-ctr-prediction/sampleSubmission.gz')
print('Train dataset:',train.shape)
print('Test dataset:',test.shape)
print('Submission:',submission.shape)

Train dataset: (500000, 24)
Test dataset: (4577464, 23)
Submission: (4577464, 2)


In [4]:
train.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,10006777279679619273,0,2014-10-21,1005,0,85f751fd,c4e18dd6,50e219e0,d2bb6502,2347f47a,8ded1f7a,4b2309e9,22c2dcf4,d6e0e6ff,1,3,18987,320,50,2158,3,291,100193,61
1,10007944429976961145,1,2014-10-21,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,07875ea4,aaffed8f,1,0,15701,320,50,1722,0,35,-1,79
2,10010504760200486071,0,2014-10-21,1005,1,5ee41ff2,17d996e6,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,d012a1cb,ecb851b2,1,0,16615,320,50,1863,3,39,100188,23
3,10011658782619041235,1,2014-10-21,1005,0,0aab7161,660aeadc,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,0086332e,1f0bc64f,1,0,15699,320,50,1722,0,35,-1,79
4,10018283997673108970,0,2014-10-21,1002,0,75f43c5a,c4e18dd6,50e219e0,ecad2386,7801e8d9,07d7df22,03cb167d,3414a157,cf6e17c1,0,0,21665,320,50,2493,3,35,-1,117


### sparse feature
 * C1
 * banner_pos
 * site_id
 * site_domain
 * site_category
 * app_id
 * app_domain
 * app_category
 * device_id
 * device_ip
 * device_model
 * device_type
 * device_conn_type

### Dense feature
 * C14 ~ c21

In [5]:
# !pip install deepctr_torch

In [6]:
import torch
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names

In [7]:
train

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,10006777279679619273,0,2014-10-21 00:00:00,1005,0,85f751fd,c4e18dd6,50e219e0,d2bb6502,2347f47a,8ded1f7a,4b2309e9,22c2dcf4,d6e0e6ff,1,3,18987,320,50,2158,3,291,100193,61
1,10007944429976961145,1,2014-10-21 00:00:00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,07875ea4,aaffed8f,1,0,15701,320,50,1722,0,35,-1,79
2,10010504760200486071,0,2014-10-21 00:00:00,1005,1,5ee41ff2,17d996e6,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,d012a1cb,ecb851b2,1,0,16615,320,50,1863,3,39,100188,23
3,10011658782619041235,1,2014-10-21 00:00:00,1005,0,0aab7161,660aeadc,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,0086332e,1f0bc64f,1,0,15699,320,50,1722,0,35,-1,79
4,10018283997673108970,0,2014-10-21 00:00:00,1002,0,75f43c5a,c4e18dd6,50e219e0,ecad2386,7801e8d9,07d7df22,03cb167d,3414a157,cf6e17c1,0,0,21665,320,50,2493,3,35,-1,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,9944737828120924015,1,2014-10-30 23:00:00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,2e93a860,5dee1d65,1,0,22676,320,50,2616,0,35,100083,51
499996,9953528129742104513,1,2014-10-30 23:00:00,1005,0,6c7e709c,246937e7,3e814130,ecad2386,7801e8d9,07d7df22,a99f214a,cda3d410,ee007ad3,1,0,23725,320,50,2716,3,47,-1,23
499997,9971338005456236690,0,2014-10-30 23:00:00,1005,0,85f751fd,c4e18dd6,50e219e0,d36838b1,0e8616ad,0f2161f8,a99f214a,7818404b,c01fd4e5,1,2,23866,320,50,2736,0,33,100170,246
499998,9998515968748286661,0,2014-10-30 23:00:00,1005,1,856e6d3f,58a89a43,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,44e7d8bd,be6db1d7,1,0,23997,320,50,2748,0,35,-1,79


In [8]:
train.iloc[::,2:14]

,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model
0,2014-10-21 00:00:00,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,905d2fbc,1b13b020
1,2014-10-21 00:00:00,1005,0,85f751fd,c4e18dd6,50e219e0,0acbeaa3,45a51db4,f95efa07,a99f214a,061893d4,68b900d9
2,2014-10-21 00:00:00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,fab514ae,4ceb2e0b
3,2014-10-21 00:00:00,1005,0,85f751fd,c4e18dd6,50e219e0,febd1138,82e27996,0f2161f8,a99f214a,ff8bfff2,aad45b01
4,2014-10-21 00:00:00,1005,0,85f751fd,c4e18dd6,50e219e0,5e3f096f,2347f47a,0f2161f8,a99f214a,d5f8b9cd,2cd8ff6d
...,...,...,...,...,...,...,...,...,...,...,...,...
499995,2014-10-30 23:00:00,1005,1,cbb01868,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,a5caf458,15b57b37
499996,2014-10-30 23:00:00,1005,0,83a0ad1a,5c9ae867,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,e814a982,d4897fef
499997,2014-10-30 23:00:00,1002,0,629fa55d,aad05c18,50e219e0,ecad2386,7801e8d9,07d7df22,c1d4953a,3c4fb380,08ac11ab
499998,2014-10-30 23:00:00,1005,1,b7e9786d,b12b9f85,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,e1b5bcff,4ec1036c


In [9]:
sparse_features = train.iloc[::,2:16].columns.values.tolist()
dense_features = train.iloc[::,16:].columns.values.tolist()
target = ['click']

In [10]:
print(test.columns, len(test.columns))

Index(['id', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object') 23


In [11]:
print(train.columns, len(train.columns))

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object') 24


In [12]:
train

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,10009147085943364421,0,2014-10-21 00:00:00,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,905d2fbc,1b13b020,1,0,17037,320,50,1934,2,39,-1,16
1,10010966574628106108,1,2014-10-21 00:00:00,1005,0,85f751fd,c4e18dd6,50e219e0,0acbeaa3,45a51db4,f95efa07,a99f214a,061893d4,68b900d9,1,0,20596,320,50,2161,0,35,100034,157
2,10040563266286189835,0,2014-10-21 00:00:00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,fab514ae,4ceb2e0b,1,0,15703,320,50,1722,0,35,-1,79
3,10075942292746331345,0,2014-10-21 00:00:00,1005,0,85f751fd,c4e18dd6,50e219e0,febd1138,82e27996,0f2161f8,a99f214a,ff8bfff2,aad45b01,1,0,21189,320,50,2424,1,161,100193,71
4,10084015716773008131,0,2014-10-21 00:00:00,1005,0,85f751fd,c4e18dd6,50e219e0,5e3f096f,2347f47a,0f2161f8,a99f214a,d5f8b9cd,2cd8ff6d,1,0,18993,320,50,2161,0,35,-1,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,9913583710066759728,0,2014-10-30 23:00:00,1005,1,cbb01868,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,a5caf458,15b57b37,1,2,17037,320,50,1934,2,39,100101,16
499996,9937812503233592875,0,2014-10-30 23:00:00,1005,0,83a0ad1a,5c9ae867,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,e814a982,d4897fef,1,0,19950,320,50,1800,3,167,100077,23
499997,9951974778276958726,1,2014-10-30 23:00:00,1002,0,629fa55d,aad05c18,50e219e0,ecad2386,7801e8d9,07d7df22,c1d4953a,3c4fb380,08ac11ab,0,0,18858,320,50,1882,3,35,-1,13
499998,9995851231658276345,1,2014-10-30 23:00:00,1005,1,b7e9786d,b12b9f85,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,e1b5bcff,4ec1036c,1,0,16858,320,50,1887,3,39,100199,23


In [13]:
def new_hour(x):
    try:
        return x.hour
    except Exception:
        return 0

In [14]:
test.hour = test.hour.apply(new_hour)
train.hour = train.hour.apply(new_hour)

In [15]:
train

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,10009147085943364421,0,0,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,905d2fbc,1b13b020,1,0,17037,320,50,1934,2,39,-1,16
1,10010966574628106108,1,0,1005,0,85f751fd,c4e18dd6,50e219e0,0acbeaa3,45a51db4,f95efa07,a99f214a,061893d4,68b900d9,1,0,20596,320,50,2161,0,35,100034,157
2,10040563266286189835,0,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,fab514ae,4ceb2e0b,1,0,15703,320,50,1722,0,35,-1,79
3,10075942292746331345,0,0,1005,0,85f751fd,c4e18dd6,50e219e0,febd1138,82e27996,0f2161f8,a99f214a,ff8bfff2,aad45b01,1,0,21189,320,50,2424,1,161,100193,71
4,10084015716773008131,0,0,1005,0,85f751fd,c4e18dd6,50e219e0,5e3f096f,2347f47a,0f2161f8,a99f214a,d5f8b9cd,2cd8ff6d,1,0,18993,320,50,2161,0,35,-1,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,9913583710066759728,0,23,1005,1,cbb01868,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,a5caf458,15b57b37,1,2,17037,320,50,1934,2,39,100101,16
499996,9937812503233592875,0,23,1005,0,83a0ad1a,5c9ae867,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,e814a982,d4897fef,1,0,19950,320,50,1800,3,167,100077,23
499997,9951974778276958726,1,23,1002,0,629fa55d,aad05c18,50e219e0,ecad2386,7801e8d9,07d7df22,c1d4953a,3c4fb380,08ac11ab,0,0,18858,320,50,1882,3,35,-1,13
499998,9995851231658276345,1,23,1005,1,b7e9786d,b12b9f85,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,e1b5bcff,4ec1036c,1,0,16858,320,50,1887,3,39,100199,23


In [16]:
test.head()

,id,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000017e+19,0,1005,0,235ba823,f6ebf28e,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,69f45779,0eb711ec,1,0,8330,320,50,761,3,175,100075,23
1,1.000018e+19,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,e8d44657,ecb851b2,1,0,22676,320,50,2616,0,35,100083,51
2,1.000055e+19,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,10fb085b,1f0bc64f,1,0,22676,320,50,2616,0,35,100083,51
3,1.000109e+19,0,1005,0,85f751fd,c4e18dd6,50e219e0,51cedd4e,aefc06bd,0f2161f8,a99f214a,422d257a,542422a7,1,0,18648,320,50,1092,3,809,100156,61
4,1.000138e+19,0,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,2347f47a,f95efa07,a99f214a,078c6b38,1f0bc64f,1,0,23160,320,50,2667,0,47,-1,221


In [17]:
from deepctr_torch.models import DIEN

In [18]:
test_sparse_features = test.iloc[::,1:15].columns.values.tolist()
test_dense_features = test.iloc[::,15:].columns.values.tolist()

In [19]:
sparse_features

['hour',
 'C1',
 'banner_pos',
 'site_id',
 'site_domain',
 'site_category',
 'app_id',
 'app_domain',
 'app_category',
 'device_id',
 'device_ip',
 'device_model',
 'device_type',
 'device_conn_type']

In [20]:
test_dense_features

['C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']

In [21]:
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
scaler = MinMaxScaler(feature_range=(0,1))
encoder = LabelEncoder()

In [22]:
for feat in test_sparse_features:
    print(feat)
    encoder.fit(pd.concat([train[feat], test[feat]]))
    train[feat] = encoder.transform(train[feat])
    test[feat] = encoder.transform(test[feat])
scaler.fit(pd.concat([train[dense_features], test[dense_features]]))
train[dense_features] = scaler.transform(train[dense_features])
test[dense_features] = scaler.transform(test[test_dense_features])

hour
C1
banner_pos
site_id
site_domain
site_category
app_id
app_domain
app_category
device_id
device_ip
device_model
device_type
device_conn_type


In [23]:
test

,id,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000017e+19,0,2,0,436,3637,20,4271,115,0,241375,541576,338,1,0,0.331818,0.221239,0.029880,0.242074,1.000000,0.078627,0.998274,0.086614
1,1.000018e+19,0,2,0,393,3593,1,4271,115,0,241375,1190817,5364,1,0,0.930216,0.221239,0.029880,0.933980,0.000000,0.001107,0.998354,0.196850
2,1.000055e+19,0,2,0,393,3593,1,4271,115,0,241375,86778,677,1,0,0.930216,0.221239,0.029880,0.933980,0.000000,0.001107,0.998354,0.196850
3,1.000109e+19,0,2,0,1686,2920,5,1450,169,4,241375,337872,1894,1,0,0.762201,0.221239,0.029880,0.365535,1.000000,0.429679,0.999082,0.236220
4,1.000138e+19,0,2,0,1686,2920,5,2823,27,27,241375,38600,677,1,0,0.950405,0.221239,0.029880,0.953003,0.000000,0.007752,0.000000,0.866142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4577459,9.998167e+18,23,2,0,1856,1789,3,4271,115,0,241375,965248,3013,1,0,0.720739,0.199115,0.229084,0.701977,0.666667,0.003322,0.000000,0.125984
4577460,9.998250e+18,23,2,0,295,3593,20,4271,115,0,241375,163048,4875,1,0,0.906357,0.221239,0.029880,0.907497,0.000000,0.220377,0.998364,0.866142
4577461,9.998802e+16,23,2,0,1188,1789,3,4271,115,0,241375,981417,4722,1,0,0.720739,0.199115,0.229084,0.701977,0.666667,0.003322,0.000000,0.125984
4577462,9.999087e+18,23,2,0,1686,2920,5,2654,168,4,241375,526055,120,1,2,0.979478,0.221239,0.029880,0.977993,0.333333,0.078627,0.999411,0.275591


In [24]:
train_test = pd.concat([train.drop(columns=['id']), test.drop(columns=['id'])])

In [25]:
fixlen_feature_columns = [SparseFeat(feat, train_test[feat].nunique()) for feat in sparse_features]
fixlen_feature_columns += [DenseFeat(feat,1,) for feat in dense_features]

In [26]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

In [27]:
dnn_feature_columns

[SparseFeat(name='hour', vocabulary_size=24, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='hour', group_name='default_group'),
 SparseFeat(name='C1', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C1', group_name='default_group'),
 SparseFeat(name='banner_pos', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='banner_pos', group_name='default_group'),
 SparseFeat(name='site_id', vocabulary_size=3193, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_id', group_name='default_group'),
 SparseFeat(name='site_domain', vocabulary_size=3780, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_domain', group_name='default_group'),
 SparseFeat(name='site_category', vocabulary_size=22, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_category', group_name='default_group'),
 SparseFeat(name='app_id', vocabulary_size=4628, embedding_dim=4, use_hash=False, 

In [28]:
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
feature_names

['hour',
 'C1',
 'banner_pos',
 'site_id',
 'site_domain',
 'site_category',
 'app_id',
 'app_domain',
 'app_category',
 'device_id',
 'device_ip',
 'device_model',
 'device_type',
 'device_conn_type',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21']

In [29]:
train_model_input = {name: train[name].values for name in feature_names}

In [30]:
train_model_input

{'hour': array([ 0,  0,  0, ..., 23, 23, 23]),
 'C1': array([2, 2, 2, ..., 1, 2, 2]),
 'banner_pos': array([1, 0, 0, ..., 0, 1, 0]),
 'site_id': array([2799, 1686,  393, ..., 1295, 2290,  393]),
 'site_domain': array([1895, 2920, 3593, ..., 2576, 2652, 3593]),
 'site_category': array([20,  5,  1, ...,  5, 20,  1]),
 'app_id': array([4271,  205, 4271, ..., 4271, 4271, 4271]),
 'app_domain': array([115,  64, 115, ..., 115, 115, 115]),
 'app_category': array([ 0, 27,  0, ...,  0,  0,  0]),
 'device_id': array([241375, 241375, 241375, ..., 275966, 241375, 241375]),
 'device_ip': array([ 738239,   31203, 1282014, ...,  307764, 1154279,  281613]),
 'device_model': array([ 588, 2355, 1737, ...,  189, 1767, 2547]),
 'device_type': array([1, 1, 1, ..., 0, 1, 1]),
 'device_conn_type': array([0, 0, 0, ..., 0, 0, 0]),
 'C14': array([0.69500292, 0.84345541, 0.63935931, ..., 0.77096021, 0.6875365 ,
        0.9127388 ]),
 'C15': array([0.22123894, 0.22123894, 0.22123894, ..., 0.22123894, 0.22123894,


In [31]:
test_model_input = {name: test[name].values for name in feature_names}

In [32]:
test_model_input

{'hour': array([ 0,  0,  0, ..., 23, 23, 23]),
 'C1': array([2, 2, 2, ..., 2, 2, 2]),
 'banner_pos': array([0, 0, 0, ..., 0, 0, 0]),
 'site_id': array([ 436,  393,  393, ..., 1188, 1686,  295]),
 'site_domain': array([3637, 3593, 3593, ..., 1789, 2920, 3593]),
 'site_category': array([20,  1,  1, ...,  3,  5, 20]),
 'app_id': array([4271, 4271, 4271, ..., 4271, 2654, 4271]),
 'app_domain': array([115, 115, 115, ..., 115, 168, 115]),
 'app_category': array([0, 0, 0, ..., 0, 4, 0]),
 'device_id': array([241375, 241375, 241375, ..., 241375, 241375, 241375]),
 'device_ip': array([ 541576, 1190817,   86778, ...,  981417,  526055,   94369]),
 'device_model': array([ 338, 5364,  677, ..., 4722,  120, 3641]),
 'device_type': array([1, 1, 1, ..., 1, 1, 1]),
 'device_conn_type': array([0, 0, 0, ..., 0, 2, 0]),
 'C14': array([0.3318178 , 0.93021607, 0.93021607, ..., 0.72073913, 0.97947777,
        0.91261366]),
 'C15': array([0.22123894, 0.22123894, 0.22123894, ..., 0.19911504, 0.22123894,
      

In [33]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [34]:
from torch.optim import AdamW

In [35]:
1

1

# PLE

#### 0.4075

In [169]:
from deepctr_torch.models import PLE

In [191]:
model = PLE(dnn_feature_columns,device=device,dnn_dropout=0.7)

In [192]:
optim = AdamW(params=model.parameters(), lr=1e-4)

In [193]:
model.compile(optimizer = optim,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy','auc'])

In [194]:
from torch.nn.functional import one_hot

In [195]:
y = one_hot(torch.tensor(train[target].values)).numpy().squeeze(1)

In [196]:
%%time
history = model.fit(train_model_input, y, batch_size=1024, epochs=3, verbose=1,
                        validation_split=0.2)

cpu
Train on 400000 samples, validate on 100000 samples, 391 steps per epoch


391it [00:36, 10.63it/s]


Epoch 1/3
37s - loss:  1.0488 - binary_crossentropy:  0.5181 - auc:  0.5215 - val_binary_crossentropy:  0.4269 - val_auc:  0.6544


391it [00:31, 12.39it/s]


Epoch 2/3
32s - loss:  0.8490 - binary_crossentropy:  0.4244 - auc:  0.6985 - val_binary_crossentropy:  0.4012 - val_auc:  0.7170


391it [00:28, 13.77it/s]


Epoch 3/3
29s - loss:  0.7949 - binary_crossentropy:  0.3973 - auc:  0.7590 - val_binary_crossentropy:  0.3984 - val_auc:  0.7269
CPU times: user 3min 11s, sys: 1min 12s, total: 4min 23s
Wall time: 1min 40s


In [197]:
pred_ans = model.predict(test_model_input, 1024)

In [198]:
submission['click'] = pred_ans[:, 1]
submission.to_csv('submission.csv',index=False)

# Deep FM

#### 0.4068

In [200]:
from deepctr_torch.models import DeepFM

In [201]:
model = DeepFM(linear_feature_columns, dnn_feature_columns,device=device,dnn_dropout=0.7)

In [202]:
optim = AdamW(params=model.parameters(), lr=1e-4)

In [203]:
model.compile(optimizer = optim,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy','auc'])

In [204]:
%%time
history = model.fit(train_model_input, train[target].values, batch_size=1024, epochs=3, verbose=1,
                        validation_split=0.2)

cpu
Train on 400000 samples, validate on 100000 samples, 391 steps per epoch


391it [00:30, 12.76it/s]


Epoch 1/3
31s - loss:  0.5131 - binary_crossentropy:  0.5130 - auc:  0.5365 - val_binary_crossentropy:  0.4278 - val_auc:  0.6596


391it [00:27, 14.30it/s]


Epoch 2/3
27s - loss:  0.4304 - binary_crossentropy:  0.4304 - auc:  0.6927 - val_binary_crossentropy:  0.4097 - val_auc:  0.6970


391it [00:27, 14.13it/s]


Epoch 3/3
28s - loss:  0.4100 - binary_crossentropy:  0.4100 - auc:  0.7320 - val_binary_crossentropy:  0.3986 - val_auc:  0.7231
CPU times: user 3min, sys: 1min 12s, total: 4min 12s
Wall time: 1min 27s


In [205]:
pred_ans = model.predict(test_model_input, 1024)

In [206]:
submission['click'] = pred_ans
submission.to_csv('submission.csv',index=False)

# CatBoostRegressor

#### 0.4102

In [100]:
from catboost import CatBoostRegressor

In [107]:
train

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,10009147085943364421,0,0,2,1,2799,1895,20,4271,115,0,241375,738239,588,1,0,0.695003,0.221239,0.02988,0.679597,0.666667,0.003322,0.000000,0.059055
1,10010966574628106108,1,0,2,0,1686,2920,5,205,64,27,241375,31203,2355,1,0,0.843455,0.221239,0.02988,0.764267,0.000000,0.001107,0.997865,0.614173
2,10040563266286189835,0,0,2,0,393,3593,1,4271,115,0,241375,1282014,1737,1,0,0.639359,0.221239,0.02988,0.600522,0.000000,0.001107,0.000000,0.307087
3,10075942292746331345,0,0,2,0,1686,2920,5,4607,129,4,241375,1306927,3866,1,0,0.868191,0.221239,0.02988,0.862365,0.333333,0.070875,0.999451,0.275591
4,10084015716773008131,0,0,2,0,1686,2920,5,1705,27,4,241375,1094178,974,1,0,0.776591,0.221239,0.02988,0.764267,0.000000,0.001107,0.000000,0.614173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,9913583710066759728,0,23,2,1,2529,1895,20,4271,115,0,241375,847438,460,1,1,0.695003,0.221239,0.02988,0.679597,0.666667,0.003322,0.998534,0.059055
499996,9937812503233592875,0,23,2,0,1661,1388,20,4271,115,0,241375,1186959,4814,1,0,0.816510,0.221239,0.02988,0.629616,1.000000,0.074197,0.998294,0.086614
499997,9951974778276958726,1,23,1,0,1295,2576,5,4271,115,0,275966,307764,189,0,0,0.770960,0.221239,0.02988,0.660201,1.000000,0.001107,0.000000,0.047244
499998,9995851231658276345,1,23,2,1,2290,2652,20,4271,115,0,241375,1154279,1767,1,0,0.687536,0.221239,0.02988,0.662066,1.000000,0.003322,0.999511,0.086614


In [105]:
model = CatBoostRegressor(cat_features=[x for x in train.columns if x in ['C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']], iterations=200)

In [106]:
model.fit(X=train.drop(columns=['click', 'id']), y=train['click'], plot=True, verbose=False,)

CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=14]=0.6950029198298157 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [238]:
out = model.predict(test.drop(columns=['id']))

In [239]:
submission['click'] = out

In [240]:
submission.to_csv('submission.csv',index=False)

In [42]:
sparse_features

['hour',
 'C1',
 'banner_pos',
 'site_id',
 'site_domain',
 'site_category',
 'app_id',
 'app_domain',
 'app_category',
 'device_id',
 'device_ip',
 'device_model',
 'device_type',
 'device_conn_type']

# AFM

In [99]:
from deepctr_torch.models import 

ImportError: cannot import name 'edcn' from 'deepctr_torch.models' (/Users/bolevard/.pyenv/versions/3.8.13/envs/3_8_jupyter/lib/python3.8/site-packages/deepctr_torch/models/__init__.py)

In [91]:
new_dnn_feature_columns = [x for x in dnn_feature_columns][:14]

In [92]:
model = AFM(linear_feature_columns, new_dnn_feature_columns, sparse_features, device=device)

In [93]:
optim = AdamW(params=model.parameters(), lr=1e-4)

In [94]:
model.compile(optimizer = optim,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy','auc'])

In [95]:
train_model_input

{'hour': array([ 0,  0,  0, ..., 23, 23, 23]),
 'C1': array([2, 2, 2, ..., 1, 2, 2]),
 'banner_pos': array([1, 0, 0, ..., 0, 1, 0]),
 'site_id': array([2799, 1686,  393, ..., 1295, 2290,  393]),
 'site_domain': array([1895, 2920, 3593, ..., 2576, 2652, 3593]),
 'site_category': array([20,  5,  1, ...,  5, 20,  1]),
 'app_id': array([4271,  205, 4271, ..., 4271, 4271, 4271]),
 'app_domain': array([115,  64, 115, ..., 115, 115, 115]),
 'app_category': array([ 0, 27,  0, ...,  0,  0,  0]),
 'device_id': array([241375, 241375, 241375, ..., 275966, 241375, 241375]),
 'device_ip': array([ 738239,   31203, 1282014, ...,  307764, 1154279,  281613]),
 'device_model': array([ 588, 2355, 1737, ...,  189, 1767, 2547]),
 'device_type': array([1, 1, 1, ..., 0, 1, 1]),
 'device_conn_type': array([0, 0, 0, ..., 0, 0, 0]),
 'C14': array([0.69500292, 0.84345541, 0.63935931, ..., 0.77096021, 0.6875365 ,
        0.9127388 ]),
 'C15': array([0.22123894, 0.22123894, 0.22123894, ..., 0.22123894, 0.22123894,


In [96]:
%%time
history = model.fit(train_model_input, train[target].values, batch_size=1024, epochs=3, verbose=1,
                        validation_split=0.2)

cpu
Train on 400000 samples, validate on 100000 samples, 391 steps per epoch


391it [00:29, 13.13it/s]


Epoch 1/3
30s - loss:  0.6091 - binary_crossentropy:  0.6091 - auc:  0.5876 - val_binary_crossentropy:  0.5380 - val_auc:  0.5913


391it [00:28, 13.63it/s]


Epoch 2/3
29s - loss:  0.5076 - binary_crossentropy:  0.5076 - auc:  0.6320 - val_binary_crossentropy:  0.4717 - val_auc:  0.6195


391it [00:29, 13.30it/s]


Epoch 3/3
30s - loss:  0.4657 - binary_crossentropy:  0.4657 - auc:  0.6648 - val_binary_crossentropy:  0.4451 - val_auc:  0.6480
CPU times: user 3min 16s, sys: 1min 13s, total: 4min 30s
Wall time: 1min 30s


In [47]:
help(model.fit)

Help on method fit in module deepctr_torch.models.basemodel:

fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, initial_epoch=0, validation_split=0.0, validation_data=None, shuffle=True, callbacks=None) method of deepctr_torch.models.din.DIN instance
    :param x: Numpy array of training data (if the model has a single input), or list of Numpy arrays (if the model has multiple inputs).If input layers in the model are named, you can also pass a
        dictionary mapping input names to Numpy arrays.
    :param y: Numpy array of target (label) data (if the model has a single output), or list of Numpy arrays (if the model has multiple outputs).
    :param batch_size: Integer or `None`. Number of samples per gradient update. If unspecified, `batch_size` will default to 256.
    :param epochs: Integer. Number of epochs to train the model. An epoch is an iteration over the entire `x` and `y` data provided. Note that in conjunction with `initial_epoch`, `epochs` is to be understood as

In [97]:
pred_ans = model.predict(test_model_input, 1024)

In [98]:
submission['click'] = pred_ans
submission.to_csv('submission.csv',index=False)